# Preprocessing Data

## Import Library

In [10]:
import os
import numpy as np
import pandas as pd
from sktime.datasets import load_from_tsfile_to_dataframe


## Path

In [11]:
TRAIN_DIR = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Klasifikasi_Colposcopy\Dataset\Colposcopy_TRAIN.ts"
TEST_DIR = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Klasifikasi_Colposcopy\Dataset\Colposcopy_TEST.ts"

X_train, y_train = load_from_tsfile_to_dataframe(TRAIN_DIR)
X_test,  y_test  = load_from_tsfile_to_dataframe(TEST_DIR)

print("Train path :", TRAIN_DIR)
print("Val path   :", TEST_DIR)

Train path : D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Klasifikasi_Colposcopy\Dataset\Colposcopy_TRAIN.ts
Val path   : D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Klasifikasi_Colposcopy\Dataset\Colposcopy_TEST.ts


## Gabungkan Data

In [12]:
X_all = np.vstack((X_train, X_test))   

y_all = np.concatenate((y_train, y_test))  

print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)
print("Distribusi kelas:", np.unique(y_all, return_counts=True))

X_all shape: (200, 1)
y_all shape: (200,)
Distribusi kelas: (array(['0', '1', '2', '3', '4', '5'], dtype='<U1'), array([15, 24, 20, 48, 37, 56], dtype=int64))


tidak ada data kosong(missing value) di data train maupun test

## Split Data

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_all,
    y_all,
    test_size=0.2,
    stratify=y_all,
    random_state=42
)

## Normalisasi Data

In [16]:
def minmax_norm_array_object(X):
    X_norm = np.empty_like(X, dtype=object)

    for i in range(X.shape[0]):
        ts = np.asarray(X[i, 0], dtype=float)
        min_v = ts.min()
        max_v = ts.max()

        if max_v - min_v != 0:
            X_norm[i, 0] = (ts - min_v) / (max_v - min_v)
        else:
            X_norm[i, 0] = np.zeros_like(ts)

    return X_norm


X_train_norm = minmax_norm_array_object(X_train)
X_test_norm  = minmax_norm_array_object(X_test)

print("X_train_norm shape:", X_train_norm.shape)
print("X_test_norm shape :", X_test_norm.shape)


X_train_norm shape: (160, 1)
X_test_norm shape : (40, 1)


fit normalisasi hanya digunakan pada data train,data test hanya di transform

## Reshape Data Time series ke Format LSTM

In [17]:
def nested_to_lstm_input(X_nested):
      
    X = np.vstack(X_nested[:, 0])

    X = X[..., np.newaxis]

    return X


In [18]:
X_train_lstm = nested_to_lstm_input(X_train_norm)
X_test_lstm  = nested_to_lstm_input(X_test_norm)

print("X_train_lstm shape:", X_train_lstm.shape)
print("X_test_lstm shape :", X_test_lstm.shape)


X_train_lstm shape: (160, 180, 1)
X_test_lstm shape : (40, 180, 1)


# Label

In [19]:
y_train_final = y_train
y_test_final  = y_test

In [20]:
import numpy as np

np.savez(
    "colposcopy_lstm_data.npz",
    X_train_lstm=X_train_lstm,
    y_train=y_train,
    X_test_lstm=X_test_lstm,
    y_test=y_test
)

print("Data preprocessing berhasil disimpan")


Data preprocessing berhasil disimpan
